# Trigger efficiency

In [7]:
import os
import ipywidgets as widgets
import h5py
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats

from math import sqrt
from glob import glob
from ipywidgets import interact, interact_manual
from tqdm import tqdm_notebook as tqdm
from shapely.geometry import MultiLineString, LineString
from collections.abc import Iterable
from matplotlib.colors import LogNorm

In [8]:
directory = "/global/project/projectdirs/dune/data/larpix/processed_data/prod_20_10_18/"
filepaths = glob(directory+"*.h5")
dirs = list(map(os.path.dirname, filepaths))
filenames = list(map(os.path.basename, filepaths))
file_dict = dict(zip(filenames,dirs))

In [9]:
pix_x_min, pix_x_max = -(4.434 * 69) / 2 - 4.434/2, +(4.434 * 69) / 2 + 4.434/2
pix_y_min, pix_y_max = -(4.434 * 69) / 2 - 4.434/2, +(4.434 * 69) / 2 + 4.434/2
n = 71
x_bins, dx = np.linspace(pix_x_min, pix_x_max, n, retstep=True)
y_bins, dy = np.linspace(pix_y_min, pix_y_max, n, retstep=True)

lines = []
for x in x_bins:
    lines.append(((x, pix_y_min), (x, pix_y_max)))
for y in y_bins:
    lines.append(((pix_x_min, y), (pix_x_max, y)))
    
grid = MultiLineString(lines)

MeVToElectrons = 4.237e+04
Ab = 0.800
kb = 0.0486
eField = 0.50 # kV/cm
lArDensity = 1.38 # g/cm^3
dEdxMIP = 2.12 # MeV/cm

In [10]:
def profile_plot(x,y,xbins,ax,statistic='mean',std_on=True,title='',fmt='r.'):
    binned = scipy.stats.binned_statistic(x, y, bins=xbins, statistic=statistic)
    binned_count = scipy.stats.binned_statistic(x, y, bins=xbins, statistic='count')
    binned_std = scipy.stats.binned_statistic(x, y, bins=xbins, statistic='std')
    value = binned.statistic
    std = binned_std.statistic/np.sqrt(binned_count.statistic)
    bin_edges = binned.bin_edges
    bin_centers = (bin_edges[:-1] + bin_edges[1:])/2.
    ax.errorbar(x=bin_centers, y=value, yerr=std, fmt=fmt,label=title,ms=4)
    
def mpv(x,range=(-2,5)):
    val,bins = np.histogram(x,bins=round(sqrt(len(x))),range=range)
    idx = np.argmax(val)
    return (bins[idx] + bins[idx+1])/2

In [13]:
%matplotlib widget

@interact_manual
def test(filenames=widgets.SelectMultiple(options=filenames,
                                          value=[filenames[0]],
                                          description="File", 
                                          layout=widgets.Layout(width='35%'))):
    
    num_eff_pid = np.zeros((72,72))
    den_eff_pid = np.zeros((72,72))
    num_eff = []
    den_eff = []
    num_eff_costheta = []
    den_eff_costheta = []
    meas = []
    exp = []

    for filename in filenames:
        
        f = h5py.File(file_dict[filename]+"/"+filename,'r')
        tracks = f['tracks']
        
        if not len(tracks):
            print("No events available in", filename)
            return

        mask = tracks['start'][:,2] < 300
        z_scale = f['tracks'].attrs['z_scale']
            
        for t in tqdm(tracks[mask],desc='Processing tracks...'):
            expected_charge = {}
            measured_charge = {}
            hit_ref = t['hit_ref']
            evd_ref = t['event_ref']
            event_idx = f['events'][evd_ref]['evid'][0]
            
            hit_rel_ts = f['hits'][hit_ref]['ts'] - f['events'][event_idx]['ts_start']
            hit_q = f['hits'][hit_ref]['q']
            hit_x = f['hits'][hit_ref]['px'] 
            hit_y = f['hits'][hit_ref]['py'] 
            hit_z = hit_rel_ts * z_scale
            
            for i in range(len(hit_x)):
                x = hit_x[i]
                y = hit_y[i]
                q = hit_q[i]
                pix_id = (np.digitize(x, x_bins), np.digitize(y, y_bins))
                if pix_id in measured_charge:
                    measured_charge[pix_id] += q
                else:
                    measured_charge[pix_id] = q

            z1 = t['start'][2] * z_scale
            z2 = t['end'][2] * z_scale
            y1 = t['start'][1]
            y2 = t['end'][1]
            x1 = t['start'][0]
            x2 = t['end'][0]
            track_length = np.sqrt((x2-x1)**2+(y2-y1)**2+(z2-z1)**2)
            xy_length = np.sqrt((x2-x1)**2+(y2-y1)**2)
            costheta = xy_length/track_length

            line = LineString(np.c_[(x1,x2), (y1,y2)])
            segments = line.difference(grid)
            
            if not isinstance(segments, Iterable):
                segments = [segments]

            for segment in segments:
                x, y = segment.xy
                length = np.sqrt((max(x)-min(x))**2+(max(y)-min(y))**2)/costheta
                x_med = np.mean(x)
                y_med = np.mean(y)
                pix_id = (np.digitize(x_med, x_bins), np.digitize(y_med, y_bins))
                dE = length/10*dEdxMIP
                recomb = Ab / (1 + kb * dEdxMIP / (lArDensity * eField))
                expected_charge[pix_id] = dE*MeVToElectrons*recomb/1e3 
            
            for pid in expected_charge:
                den_eff_pid[pid] += 1
                den_eff.append(expected_charge[pid])
                den_eff_costheta.append(costheta)
                if pid in measured_charge:
                    num_eff.append(expected_charge[pid])
                    num_eff_pid[pid] += 1
                    num_eff_costheta.append(costheta)


            for pid in measured_charge:
                this_exp_pid = 0
                if pid in expected_charge:
                    this_exp_pid = expected_charge[pid]

                meas.append(measured_charge[pid])
                exp.append(this_exp_pid)

    fig, ax = plt.subplots(1, 3, figsize=(15,4), constrained_layout=True, num='Trigger efficiency')
    
    num_histo, bin_edges = np.histogram(num_eff, range=(0,40),bins=20)
    den_histo, bin_edges = np.histogram(den_eff, range=(0,40),bins=20)
    bin_centers = [(bin_edges[i]+bin_edges[i+1])/2. for i in range(len(bin_edges)-1)]
    eff = num_histo/den_histo
    eff_err = 1./den_histo*np.sqrt(num_histo*(1-eff))
    ax[0].errorbar(bin_centers,eff,eff_err,ls='none')
    bin_centers.insert(0, 0)
    bin_centers.append(bin_edges[-1])
    eff = np.insert(eff,0,eff[0])
    eff = np.insert(eff,-1,eff[-1])
    ax[0].step(bin_centers,eff,where='mid',c='tab:blue')
    ax[0].set_xlim(bin_edges[0],bin_edges[-1])
    ax[0].set_xlabel("$Q_{\mathrm{expected}}$ [ke]")
    ax[0].set_ylabel("Detector trigger efficiency")
    ax[0].set_ylim(0,1.05)
    
    num_histo, bin_edges_theta = np.histogram(num_eff_costheta, range=(0.5,1),bins=50)
    den_histo, bin_edges_theta = np.histogram(den_eff_costheta, range=(0.5,1),bins=50)
    bin_centers = [(bin_edges_theta[i]+bin_edges_theta[i+1])/2. for i in range(len(bin_edges_theta)-1)]
    eff = num_histo/den_histo
    eff_err = 1./den_histo*np.sqrt(num_histo*(1-eff))
    ax[1].errorbar(bin_centers,eff,eff_err,ls='none')
    bin_centers.insert(0, 0)
    bin_centers.append(bin_edges_theta[-1])
    eff = np.insert(eff,0,eff[0])
    eff = np.insert(eff,-1,eff[-1])
    ax[1].step(bin_centers,eff,where='mid',c='tab:blue')
    ax[1].set_xlim(bin_edges_theta[0],bin_edges_theta[-1])
    ax[1].set_xlabel(r"$\cos\theta$")
    ax[1].set_ylabel("Detector trigger efficiency")
    ax[1].set_ylim(0,1.05)

    eff_pid = np.nan_to_num(num_eff_pid/den_eff_pid)
    c = ax[2].imshow(eff_pid,aspect='equal',origin='lower',extent=(pix_x_min-4.434,pix_x_max+4.434,pix_y_min-4.434,pix_y_max+4.434))
    plt.colorbar(c,label='Detector trigger efficiency', aspect=40)
    ax[2].set_xlim(pix_x_min,pix_x_max)
    ax[2].set_ylim(pix_y_min,pix_y_max)
    ax[2].set_xlabel("x [cm]")
    ax[2].set_ylabel("y [cm]")
    
    fig, ax = plt.subplots(1,1)
    xbins = bin_edges
    x = np.array(exp)
    y = np.array(meas)
    h=ax.hist2d(x, y/x,bins=100,range=((0,40),(-2,5)),norm=LogNorm(vmin=1))
    profile_plot(x, y/x, xbins, ax, statistic='mean', title="Mean",fmt='rv')
    profile_plot(x, y/x, xbins, ax, statistic='median', title="Median",fmt='ro')
    profile_plot(x, y/x, xbins, ax, statistic=mpv, title="MPV",fmt='rs')
    ax.legend()
    ax.set_xlabel("$Q_{\mathrm{expected}}$ [ke]")
    ax.set_ylabel("$Q_{\mathrm{measured}}/Q_{\mathrm{expected}}$")
    plt.colorbar(h[3])
    
    fig,ax = plt.subplots(4,5,figsize=(12,10),constrained_layout=True)
    for i in range(4):
        for j in range(5):
            lim_down = (j+i*5)*2
            lim_up = lim_down+2
            ax[i][j].hist(y[(x>lim_down)&(x<lim_up)],range=(-10,250),histtype='step',lw=1,bins=52)
            ax[i][j].set_xlim(-10,250)
            ax[i][j].set_title("%i < $Q_{\mathrm{expected}}$ < %i" % (lim_down, lim_up))
            if i == 3:
                ax[i][j].set_xlabel("$Q_{\mathrm{measured}}$ [ke]")


interactive(children=(SelectMultiple(description='File', index=(0,), layout=Layout(width='35%'), options=('dat…

In [12]:
!pwd

/global/cfs/cdirs/dune/users/roberto/larpix_notebook
